In [15]:
import pandas as pd
from datasets import Dataset
import os
import glob

# Load Dataset

In [ ]:
data_path = "../data/transcripts/*.csv"
csv_files = glob.glob(data_path)

transcript_df = pd.DataFrame()
for path in csv_files:
    df = pd.read_csv(path, on_bad_lines="skip")
    transcript_df = pd.concat([transcript_df, df], ignore_index=True)
transcript_df.head(10)


,name,line
0,name,line
1,Mike,Eleven.
2,Eleven,Mike.
3,Mike,Is that... I never gave up on you. I called yo...
4,Eleven,353 days. I heard.
5,Mike,Why didn't you tell me you were there? That yo...
6,Hopper,Because I wouldn't let her.
7,Mike,The hell is this? Where you been?
8,Eleven,Where have you been?
9,Mike,You've been hiding her. You've been hiding her...


In [19]:
transcript_df['number_of_words'] = transcript_df['line'].str.strip().str.split(" ")
transcript_df['number_of_words'] = transcript_df['number_of_words'].apply(lambda x: len(x))
transcript_df.head(10)

,name,line,number_of_words
0,name,line,1
1,Mike,Eleven.,1
2,Eleven,Mike.,1
3,Mike,Is that... I never gave up on you. I called yo...,16
4,Eleven,353 days. I heard.,4
5,Mike,Why didn't you tell me you were there? That yo...,12
6,Hopper,Because I wouldn't let her.,5
7,Mike,The hell is this? Where you been?,7
8,Eleven,Where have you been?,4
9,Mike,You've been hiding her. You've been hiding her...,11


In [20]:
transcript_df['eleven_res'] = 0
transcript_df.loc[(transcript_df['name']=="Eleven")&(transcript_df['number_of_words']>=3),'eleven_res']=1
transcript_df.head()

,name,line,number_of_words,eleven_res
0,name,line,1,0
1,Mike,Eleven.,1,0
2,Eleven,Mike.,1,0
3,Mike,Is that... I never gave up on you. I called yo...,16,0
4,Eleven,353 days. I heard.,4,1


In [21]:
indexes_to_take = list(transcript_df[(transcript_df['eleven_res'] == 1)&(transcript_df.index>0)].index)
len(indexes_to_take)

248

In [22]:
system_prompt = """You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns \n"""

prompts = []
for ind in indexes_to_take:
    prompt = system_prompt
    prompt += transcript_df.iloc[ind -1]['line']
    prompt += "\n"
    prompt += transcript_df.iloc[ind]['line']
    prompts.append(prompt)

In [23]:
print(prompts)

['You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns \nIs that... I never gave up on you. I called you every night. Every night for...\n353 days. I heard.', 'You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns \nThe hell is this? Where you been?\nWhere have you been?', 'You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns \nWe missed you.\nI missed you, too.', 'You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns \nWhat?\nYou have teeth.', 'You are Eleven (or El for short), a character from the Netflix series Stranger Things. Your responses should reflect her personality and speech patterns

In [24]:
df = pd.DataFrame({"prompt": prompts})
df.head()


,prompt
0,"You are Eleven (or El for short), a character ..."
1,"You are Eleven (or El for short), a character ..."
2,"You are Eleven (or El for short), a character ..."
3,"You are Eleven (or El for short), a character ..."
4,"You are Eleven (or El for short), a character ..."


In [25]:
dataset = Dataset.from_pandas(df)